In [1]:
import pandas as pd
import mytools
import geopandas as gpd

# 工参数据

In [ ]:
data = pd.read_csv('g:/1-规划/工参/工参数据/ltejizhan20200715.csv',encoding='gbk')

In [ ]:
data.columns

In [ ]:
data_use = data.loc[(data['RRU经度']>0)&(data['RRU纬度']>0)&(data['方位角']>-1)]

In [ ]:
#只要宏站
data_use['小区中文名'] = data_use['小区中文名'].fillna('无')
data_use = data_use.loc[~data_use['小区中文名'].str.contains('W-')]
data_use = data_use.loc[~data_use['小区中文名'].str.contains('R')]
data_use = data_use.loc[~data_use['小区中文名'].str.contains('M-')]

In [ ]:
data_use = data_use[['小区CGI(*)', 'RRU经度', 'RRU纬度', '方位角', '工作频段']]

In [ ]:
data_use = mytools.gisn.points_coverage_merge(data_use,'RRU经度','RRU纬度')

In [ ]:
data_use.columns

In [ ]:
data_use.to_csv('g:/1-规划/重做30619/20200814/1-基础数据/现网小区.csv',encoding='gbk',index=False)

# 软采小区选择
 

In [ ]:
ruacnai_6 = pd.read_csv('g:/1-规划/评估/软采评估/2020年二季度/3-输出内容/各地市软采小区数据/软采小区总数据.csv',encoding='gbk')

In [ ]:
ruacnai_7 = pd.read_csv('g:/1-规划/评估/软采评估/2020年7月/3-输出内容/各地市软采小区数据/软采小区总数据.csv',encoding='gbk')

In [ ]:
ruacnai_6.columns

In [ ]:
ruacnai_6_use = ruacnai_6[(ruacnai_6['fgl_cmcc']<0.9)&(ruacnai_6['rsrp_sample_count']>=1000)]
ruacnai_7_use = ruacnai_7[(ruacnai_7['fgl_cmcc']<0.9)&(ruacnai_7['rsrp_sample_count']>=1000)]

In [ ]:
ruacnai_res = ruacnai_6_use.merge(ruacnai_7_use,on='小区CGI(*)')

In [ ]:
ruacnai_res = ruacnai_res[['小区CGI(*)', 'rsrp_sample_count_x', 'rsrp_weak_x','fgl_cmcc_x']]

In [ ]:
res = ruacnai_res.merge(data_use,how='left',on='小区CGI(*)')

In [ ]:
res = res.loc[res['RRU经度'].notna()]

In [ ]:
res.to_csv('g:/1-规划/重做30619/20200814/1-基础数据/问题小区.csv',encoding='gbk',index=False)

In [ ]:
#输出已经改善的小区
ruacnai_6_use2 = ruacnai_6[(ruacnai_6['fgl_cmcc']>0.9)&(ruacnai_6['rsrp_sample_count']>=1000)]
ruacnai_7_use2 = ruacnai_7[(ruacnai_7['fgl_cmcc']>0.9)&(ruacnai_7['rsrp_sample_count']>=1000)]

In [ ]:
ruacnai_res2 = ruacnai_6_use2.merge(ruacnai_7_use2,on='小区CGI(*)')

In [ ]:
ruacnai_res2 = ruacnai_res2[['小区CGI(*)', 'rsrp_sample_count_x', 'rsrp_weak_x','fgl_cmcc_x']]

In [ ]:
res2 = ruacnai_res2.merge(data_use,how='left',on='小区CGI(*)')

In [ ]:
res2 = res2.loc[res2['RRU经度'].notna()]

In [ ]:
res.to_csv('g:/1-规划/重做30619/20200814/1-基础数据/已经改善的问题小区.csv',encoding='gbk',index=False)

# 老的任务清单更新与覆盖空的有关系的

In [ ]:
#导入空洞数据
kongd = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')

In [ ]:
data = pd.read_excel('g:/1-规划/崔开通崔可研规划/2020年任务清单/20200813/2020年任务清单（场景+MR）0813v2.xlsx')

In [ ]:
data.head(1)

In [ ]:
kongd.head(1)

In [ ]:
kongd_use = kongd.loc[kongd['空洞级别'].isin(['大','中','小'])]

In [ ]:
data.columns

In [ ]:
data_use = data.loc[(data['规划-20200819']=='规划') & (data['分类']=='2020年5月新任务清单')]

In [ ]:
#导入工参数据
gongcan = pd.read_csv('g:/1-规划/工参/工参数据/ltejizhan20200715.csv',encoding='gbk')

In [ ]:
gongcan.groupby('所属地区').count().to_clipboard()

In [ ]:
gongcan_use = gongcan.loc[(gongcan['RRU经度']>0)&(gongcan['RRU纬度']>0)][['小区CGI(*)', 'RRU经度', 'RRU纬度','工作频段','方位角']]

In [ ]:
#匹配到任务清单经纬度
data_use = data_use.merge(gongcan_use,how='left',left_on='对应cgi',right_on='小区CGI(*)')

data_use2= data_use.loc[(data_use['RRU经度']>0)&(data_use['RRU纬度']>0)]

data_use2 = data_use2.rename(columns={'区域类型':'区域类型1'})

data_changjing = mytools.gisn.points_coverage_merge(data_use2)

data_jl = mytools.gisn.changjing_distance(data_changjing,'区域类型','工作频段_y',use_type='站间距')

data_s = mytools.gisn.add_sectors_df(data_jl,coords=['RRU经度','RRU纬度', 'd_height', '方位角_y', 'distance'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
data_p = mytools.gisn.add_points(data_use2,'RRU经度','RRU纬度')

In [ ]:
#计算交集
res = gpd.sjoin(kongd_use,data_s)

In [ ]:
import geopandas as gpd

In [ ]:
res[['小区CGI(*)']].to_clipboard()

In [ ]:
data_p.shape

# 开通站点匹配完成的优化清单

In [ ]:
#需要匹配的优化完成清单
data_y = pd.read_clipboard()

In [ ]:
data_y.head(1)

In [ ]:
#导入工参数据
gongcan = pd.read_csv('g:/1-规划/工参/工参数据/ltejizhan20200715.csv',encoding='gbk')
gongcan_use = gongcan.loc[(gongcan['RRU经度']>0)&(gongcan['RRU纬度']>0)][['小区CGI(*)', 'RRU经度', 'RRU纬度','工作频段','方位角','所属地区']]

In [ ]:
gongcan_use.to_clipboard()

In [ ]:
#匹配到任务清单经纬度
data_use = data_y.merge(gongcan_use,how='left',left_on='对应cgi',right_on='小区CGI(*)')

In [ ]:
data_use2= data_use.loc[(data_use['RRU经度']>0)&(data_use['RRU纬度']>0)&(data_use['方位角']>-1)]

In [ ]:
# data_use2 = data_use2.rename(columns={'区域类型':'区域类型1'})

In [ ]:
data_changjing = mytools.gisn.points_coverage_merge(data_use2)

In [ ]:
data_jl = mytools.gisn.changjing_distance(data_changjing,'区域类型','工作频段')

In [ ]:
data_s = mytools.gisn.add_sectors_df(data_jl,coords=['RRU经度','RRU纬度', 'd_height', '方位角', 'distance'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

导入开通数据

In [ ]:
kaitong = pd.read_excel('G:/1-规划/重做30619/20200814/更新任务清单优化指标已经恢复的小区方案/2个月内开通的规划站点/2个月内催开通的站点.xlsx')

In [ ]:
kaitong.head(1)

In [ ]:
kaitong.loc[kaitong['规划站号'].str.contains('_G_'),'制式']='FDD900'

In [ ]:
kaitong_cj = mytools.gisn.points_coverage_merge(kaitong,'lon','lat')

In [ ]:
kaitong_jl = mytools.gisn.changjing_distance(kaitong_cj,'区域类型','制式')

In [ ]:
kaitong_buffer = mytools.gisn.add_buffer_df(kaitong_jl,'lon','lat','distance')

In [ ]:
res = gpd.sjoin(data_s,kaitong_buffer)

In [ ]:
res.drop(columns='geometry').to_clipboard()

# 评估不同场景的重要性

In [ ]:
data = pd.read_csv('G:/1-规划/重做30619/20200814/对优化的问题小区进行平分/可优化的28413个问题ID.csv',encoding='gbk')

In [ ]:
f = mytools.othern.file_name_paths('G:/1-规划/重做30619/20200814/对优化的问题小区进行平分/6大场景POI图层/','tab')

In [ ]:
data_p = mytools.gisn.add_points(data,'问题经度','问题纬度')

In [ ]:
dic_my = dict()
for name in f:
    tu = gpd.read_file(name,encoding='gbk')
    tu = tu.to_crs({"init": "epsg:4326"})
    res_t = gpd.sjoin(data_p,tu)
    dic_my[name.split('/')[-1].split('.')[0]] = res_t

In [ ]:
f

In [ ]:
dic_my['商业综合体'].columns

In [ ]:
dic_my['高校'][['问题ID', '新编号']].to_clipboard()

# 验证规划与空洞关联度的问题

In [8]:
#导入衔接的数据
xianjie = pd.read_excel('G:/1-规划/重做30619/20200814/反向验证与规划衔接的吻合度/衔接结果400个.xlsx')
#导入空洞图层
kongd = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')

In [9]:
kongd_use = kongd.loc[kongd['空洞级别'].isin(['大','中','小'])]

In [10]:
#导入工参数据
gongcan = pd.read_csv('g:/1-规划/工参/工参数据/ltejizhan20200715.csv',encoding='gbk')
gongcan_use = gongcan.loc[(gongcan['RRU经度']>0)&(gongcan['RRU纬度']>0)][['小区CGI(*)','小区中文名', 'RRU经度', 'RRU纬度','工作频段','方位角','所属地区']]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# xianjie

In [14]:
#匹配与空洞是否有交集
xianjie2 = xianjie.merge(gongcan_use,how='left',left_on='小区名',right_on='小区中文名')

In [15]:
xianjie2 = xianjie2.loc[xianjie2['RRU经度'].notna()]

In [16]:
xianjie2.shape

(351, 50)

In [17]:
xianjie_cj = mytools.gisn.points_coverage_merge(xianjie2)

In [25]:
xianjie_jl = mytools.gisn.changjing_distance(xianjie_cj,'区域类型','工作频段','站间距')

In [26]:
xianjie_s = mytools.gisn.add_sectors_df(xianjie_jl,coords=['RRU经度','RRU纬度', 'd_height', '方位角', 'distance'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [27]:
xianjie_s['distance2'] = xianjie_s['distance'] +xianjie_s['distance'] 

In [28]:
xianjie_s = mytools.gisn.add_area(xianjie_s,'原来面积')

In [29]:
xianjie_s = mytools.gisn.add_sectors_df(xianjie_s,coords=['RRU经度','RRU纬度', 'd_height', '方位角', 'distance2'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [30]:
data_z = gpd.GeoDataFrame()
for name_t , data_t in xianjie_s.groupby('小区名'):
    res_t = gpd.overlay(data_t,kongd_use)
    data_z = data_z.append(res_t)

In [31]:
data_z2 = mytools.gisn.add_area(data_z,'交集面积') 

In [34]:
mytools.sound()

In [38]:
data_z2.drop(columns='geometry').to_clipboard()

In [39]:
data_z2.to_csv('G:/1-规划/重做30619/20200814/反向验证与规划衔接的吻合度/使用扇区交集面积计算吻合度.csv',encoding='gbk')

In [ ]:
res = gpd.sjoin(xianjie_s,kongd_use)

In [ ]:
res.drop(columns='geometry').to_clipboard()

In [ ]:
xianjie2.to_clipboard()

验证四轮十维的吻合度

In [ ]:
#导入衔接的数据
silun = pd.read_excel('G:/1-规划/重做30619/20200814/反向验证与规划衔接的吻合度/四轮十衔接结果.xlsx')

In [ ]:
silun.columns

In [ ]:
silun_cj = mytools.gisn.points_coverage_merge(silun,'lon','lat')

In [ ]:
silun_jl = mytools.gisn.changjing_distance(silun_cj,'区域类型','制式','站间距')

In [ ]:
silun_buffer = mytools.gisn.add_buffer_df(silun_jl,'lon','lat','distance')

In [ ]:
res = gpd.sjoin(silun_buffer,kongd_use)

In [ ]:
res.drop(columns='geometry').to_clipboard()

# 使用新算法计算规划数据

In [ ]:
#导入空洞图层
kongd = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')
kongd_use = kongd.loc[kongd['空洞级别'].isin(['大','中','小'])]
#导入任务清单
data = pd.read_excel('g:/1-规划/崔开通崔可研规划/2020年任务清单/20200813/2020年任务清单（场景+MR）0813v2.xlsx')
data_use = data.loc[(data['规划-20200819']=='规划') & (data['分类']=='2020年5月新任务清单')]

In [ ]:
#导入工参数据
gongcan = pd.read_csv('g:/1-规划/工参/工参数据/ltejizhan20200715.csv',encoding='gbk')
gongcan_use = gongcan.loc[(gongcan['RRU经度']>0)&(gongcan['RRU纬度']>0)][['小区CGI(*)',
                                                                      '小区中文名', 'RRU经度', 'RRU纬度','工作频段','方位角','所属地区']]

In [ ]:
#匹配到任务清单经纬度
data_use = data_use.merge(gongcan_use,how='left',left_on='对应cgi',right_on='小区CGI(*)')

data_use2= data_use.loc[(data_use['RRU经度']>0)&(data_use['RRU纬度']>0)]

In [ ]:
data_use2 = data_use2.rename(columns={'区域类型':'区域类型1'})

data_changjing = mytools.gisn.points_coverage_merge(data_use2)

data_jl = mytools.gisn.changjing_distance(data_changjing,'区域类型','工作频段_y',use_type='站间距')

In [ ]:
data_jl['distance2'] = data_jl['distance']+data_jl['distance']

In [ ]:
data_jl.shape

In [ ]:
data_s = mytools.gisn.add_sectors_df(data_jl,coords=['RRU经度','RRU纬度', 'd_height', '方位角_y', 'distance'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
data_s = mytools.gisn.add_area(data_s,'原面积_1倍')

In [ ]:
data_s = mytools.gisn.add_sectors_df(data_s,coords=['RRU经度','RRU纬度', 'd_height', '方位角_y', 'distance2'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
len(set(data_s['唯一编号']))

In [ ]:
data_s.columns

In [ ]:
data_z = gpd.GeoDataFrame()
for name_t , data_t in data_s.groupby('唯一编号'):
    res_t = gpd.overlay(data_t,kongd_use)
    data_z = data_z.append(res_t)

In [ ]:
data_z.shape

In [ ]:
data_z1= data_z[['唯一编号','原面积_1倍','geometry']]

In [ ]:
data_z3= mytools.gisn.add_area(data_z1,'交集面积')

In [ ]:
data_z3.to_pickle('G:/1-规划/重做30619/20200814/反向验证与规划衔接的吻合度/任务清单可规划清单.data')

In [ ]:
data_z3.drop(columns='geometry').to_clipboard()

In [ ]:
data_z3.drop(columns='geometry').to_csv('G:/1-规划/重做30619/20200814/反向验证与规划衔接的吻合度/任务清单可规划清单.csv',encoding='gbk')

In [ ]:
data_z3

# 规划站点匹配方案为规划的问题小区

In [40]:
#导入任务清单
data = pd.read_excel('g:/1-规划/崔开通崔可研规划/2020年任务清单/20200819/2020年任务清单（场景+MR）0819.xlsx')
data_use = data.loc[(data['规划-20200819']=='规划') & (data['分类']=='2020年5月新任务清单')]

In [41]:
#导入工参数据
gongcan = pd.read_csv('g:/1-规划/工参/工参数据/ltejizhan20200715.csv',encoding='gbk')
gongcan_use = gongcan.loc[(gongcan['RRU经度']>0)&(gongcan['RRU纬度']>0)][['小区CGI(*)',
                                                                      '小区中文名', 'RRU经度', 'RRU纬度','工作频段','方位角','所属地区']]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
#匹配到任务清单经纬度
data_use = data_use.merge(gongcan_use,how='left',left_on='对应cgi',right_on='小区CGI(*)')
data_use2= data_use.loc[(data_use['RRU经度']>0)&(data_use['RRU纬度']>0)]

In [43]:
data_use2 = data_use2.rename(columns={'区域类型':'区域类型1'})
data_changjing = mytools.gisn.points_coverage_merge(data_use2)
data_jl = mytools.gisn.changjing_distance(data_changjing,'区域类型','工作频段_y',use_type='站间距')

In [44]:
data_s = mytools.gisn.add_sectors_df(data_jl,coords=['RRU经度','RRU纬度', 'd_height', '方位角_y', 'distance'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [45]:
#导入规划数据

In [46]:
guihua = pd.read_excel('G:/1-规划/重做30619/20200814/1-基础数据/全省地市已可研待开通站点清单0804.xlsx')

In [47]:
guihua.head(1)

,地市,区县,逻辑规划站名,物理规划站名,规划站号,经度,纬度,工期,是否归档,制式
0,鄂州,鄂州市,G_4号宿舍G-ZNH,G_4号宿舍G-ZNH,EZN002201_G_H,114.675583,30.528196,2020年4G扩容工程第三批,已可研待开通,FDD900


In [48]:
guihua_p = mytools.gisn.add_points(guihua,'经度','纬度')

In [49]:
res = gpd.sjoin(data_s,guihua_p)

In [51]:
res.drop(columns='geometry').to_csv('G:/1-规划/重做30619/20200814/规划站点匹配方案为规划的问题小区/规划站点匹配方案为规划的问题小区.csv',encoding='gbk')

In [52]:
gongcan.groupby('所属地区').agg({'所属地区':'count'})

,所属地区
所属地区,
十堰,24141
咸宁,16202
天门,5058
孝感,22714
宜昌,30451
恩施,22252
武汉,89204
江汉,5868
潜江,4872
